# Single FID Example

This example shows an overview of the basic functionality of the Blackchirp python module. It uses data acquired using the UC Davis Ka band (26.5 - 40 GHz) CP-FTMW spectrometer.

To get started, first ensure that the module has been installed with `pip install blackchirp`. It is recommended to import the main blackchirp classes with `from blackchirp import *`. The cell below is configured to import from the development version of the module if this example notebook is run from the blackchirp source directory.


In [1]:
from blackchirp.src.blackchirp import * # replace with from blackchirp import *
from matplotlib import pyplot as plt

## Loading and Inspecting an Experiment

In this example, we will use a CP-FTMW spectrum of methyl tert-butyl ether, which is assumed to be in the directory `example-data/mtbe`. This directory contains the contents of the Blackchirp data, which was copied from the original data storage location.

In [2]:
ll example-data/mtbe/

ls: cannot access 'example-data/mtbe-2/': No such file or directory


The `BCExperiment` class is used to read in these files and construct python objects for them. Since all Blackchirp data is written in CSV format, the files are read in using `pandas.read_csv` and are available as pandas `DataFrame` objects with the same name as the corresponding CSV file. To load an experiment, pass the appropriate path to `BCExperiment`. This path should be the folder which contains the `version.csv` file for the desired experiment. Once loaded, the contents of the csv files may be inspected easily. Here we show the header.

In [3]:
exp = BCExperiment('./example-data/mtbe/')

exp.header

FileNotFoundError: Could not find Blackchirp data at /home/kncrabtree/github/blackchirp/src/python/example-data/mtbe-2

Just like in the `header.csv` file itself, each entry is associated with an `ObjKey` and a `ValueKey`, and in some cases also an `ArrayKey` and `ArrayIndex`. The combination of these 4 values can be used to select any particular row, as shown in more detail below. Each entry has a `Value` and `Units` associated with it.

For such a large DataFrame, Jupyter notebooks typically compress the output. To get a brief overview of what information is available, use the `BCExperiment.header_unique_keys()` function, which returns a set containing the unique keys in the table.

In [ ]:
exp.header_unique_keys?

In [ ]:
exp.header_unique_keys()

To view only data associated with one of these keys, use the `BCExperiment.header_rows()` function:

In [ ]:
exp.header_rows?

For instance, to obtain only the settings related to the Pulse Generator:

In [ ]:
exp.header_rows('PulseGenerator.0')

To see only the pulse widths:

In [ ]:
exp.header_rows('PulseGenerator.0','Width')

And finally, the `BCExperiment.header_value()` function retrieves one particular value from the table. A corresponding `BCExperiment.header_unit()` function can be used to retrieve the unit, if desired. Note that the value is returned as a string, so it may need to be explicitly cast to an `int` or `float` if the value will be used in a calculation.

In [ ]:
exp.header_value?

In [ ]:
exp.header_value('PulseGenerator.0','Width',3),exp.header_unit('PulseGenerator.0','Width',3)

Other csv files are similarly accessible.

In [ ]:
exp.chirps

## FID data and Fourier Transforms

The CP-FTMW data for an experiment are stored in a BCFTMW object which is accessible as `BCExperiment.ftmw`. This object can be used to view data about the available FIDs and load them from disk. For a single FID acquisition like this, the FID is loaded with `BCFTMW.get_fid()`, which returns a `BCFid` object. The `BCFID.ft()` method computes the Fourier transform of that FID. As a quick example:

In [ ]:
x,y = exp.ftmw.get_fid().ft()

fig,ax = plt.subplots(figsize=(10,3))
ax.plot(x,y)
ax.set_xlabel('Frequency (MHz)')
ax.set_xlim(26500,40000)

Taking a step back, the FID itself can be stored and visualized. The FID itself has `BCFid.y()` and `BCFid.xy()` methods which return arrays containing the FID data in units of V, and in the latter case, also the time array in units of s. For this FID, the chirp takes place from roughly 0.75 - 3.75 μs.

In [ ]:
fid = exp.ftmw.get_fid()

fidx, fidy = fid.xy()
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(fidx*1e6,fidy,label=f'{int(fid.shots)} shots')
ax.set_xlabel('Time (μs)')
ax.set_ylabel('FID (V)')
ax.legend(frameon=False)

At this point it is important to note that `fidy` is a 2D numpy array. The second axis corresponds to the frame number. In this acquisition, there is only 1 frame, but for an acquisition configured with multiple records this number may be larger. A single frame can then be selected by slicing (e.g., `fidy[:,3]`).

In [ ]:
fidy.shape

The contents of the `fidparams.csv` file are available as an attribute of the `BCFTMW` object.

In [ ]:
exp.ftmw.fidparams

This experiment contains 2 FIDs: FID 0 is the final set of data and FID 1 was a backup that was taken 15 minutes into the acquisition. The `BCFTMW.get_fid()` function takes an optional argument that specifies the FID number to retrieve. We can therefore access the backup version with `get_fid(1)`. The code below loads the backup and plots its FT together with the total, showing the decrease in noise level with increasing shots.

In [ ]:
x2,y2 = exp.ftmw.get_fid(1).ft()
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(x2,y2,label='FID 1')
ax.plot(x,y,label='FID 0')
ax.set_xlabel('Frequency (MHz)')
ax.set_xlim(36000,36010)
ax.set_ylim(0,0.15)
ax.legend()

The `BCFid.ft()` function also allows for customizing the FID processing options like those in the Blackchirp program itself. The default values are read in from `processing.csv`, which is converted into a python dictionary named `proc` which is an attribute of `BCFTMW`. One notable difference in behavior: in the python module, points within "AutoscaleMHz" of the probe frequency are set to 0 to accommodate automatic autoscaling of the plot, while in Blackchirp itself those points are still shown but are not included when computing the vertical range of the FT.

In [ ]:
exp.ftmw.proc

The `BCFid.ft()` function allows for any of these settings to be overridden.

In [ ]:
fid.ft?

For example, to apply an exponential filter and compare with the original data:

In [ ]:
expf = 5.0
x3,y3 = exp.ftmw.get_fid().ft(expf_us=expf)

fig,axes = plt.subplots(2,1,figsize=(10,6))
for ax in axes:
    ax.plot(x,y,label='No filter')
    ax.plot(x3,y3,label=f'expf = {expf:.2f} μs')
axes[0].set_xlim(26500,40000)
axes[1].set_xlim(34600,34610)
axes[1].set_ylim(0,3.5)
axes[0].legend()
axes[1].set_xlabel('Frequency (MHz)')

A greater variety of window functions is available in the Python module compared with Blackchirp. The `winf` parameter can be set to any value supported by [scipy.signal.get_window](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.get_window.html). For example, a generalized Gaussian window with variable p and σ parameters can be used by passing a tuple with the window name and parameter values:

In [ ]:
p = 1.5
sigma = len(fid)//5
x4,y4 = fid.ft(winf=('general_gaussian',p,sigma))

fig,axes = plt.subplots(2,1,figsize=(10,6))
for ax in axes:
    ax.plot(x,y,label='No window')
    ax.plot(x4,y4,label=f'Gaussian window (p={p:.2f},σ={int(sigma)})')
axes[0].set_xlim(26500,40000)
axes[1].set_xlim(34600,34610)
axes[1].set_ylim(0,3.5)
axes[0].legend()
axes[1].set_xlabel('Frequency (MHz)')

Like the FID, the FT y array is 2-dimensional, where the second axis corresponds to the frame number. By default, the FT is applied to all frames at once; if only a single frame is desired, pass its index as the `frame` parameter.

In [ ]:
y.shape